In [1]:
from numba import jit, njit
import numba as nb
import numpy as np
import pandas as pd
import time
from numba_func import *

# Numba can beat Numpy (easily)

***Illustrative example using a markov chain.***

![](https://en.wikipedia.org/wiki/Markov_chain#/media/File:Markov-cheese-lettuce-grapes.svg)

A (discrete-time) markov chain is a stochastic model with many real life applications. See the relevant [Wikipedia](https://en.wikipedia.org/wiki/Markov_chain) page for more information. The model is described in terms of
* a finite set of states
* transition probabilities from one state to another state

To simulate a markov chain, 
* set the intial state
* draw a uniform random number between 0 and 1
* compare to the cumulative transition probabilities to determine the next state
* repeat

## markov chain simulation function

### three versions
1. Native Python
1. Native Python with numby (via njit)
1. Numpy implementation

Notes:
* All functions have the same arguments
* In the first two functions, we use a digitize function for scalar, which is compiled via numba. (see numba_func.py)
* **In this case, the numba implementation is as easy to read and as short as the numpy implementation.**

In [7]:
# Nativa Python version
def markov_chain_simulation_naive(v_curr_state, v_draw, mtx_CTP0):
    """Markov chain simulation

    Args:
        v_curr_state (1-d numpy vector, integer): vector of current states
        v_draw (1-d numpy vector, float): vector of uniform random numbers between 0 and 1
        mtx_CTP0 (1-d numpy matrix, float): cumulative transition probability matrix without the last column (which consists of ones by definition)

    Returns:
        1-d numpy vector, integer: vector of next states
    """

    v_next_state = np.empty_like(v_curr_state)
    for n in range(v_next_state.size):
        curr_s = v_curr_state[n]
        v_next_state[n] = digitize_scalar_nb(v_draw[n], mtx_CTP0[curr_s])
    
    return v_next_state

# Nativa Python version (identical to above), numba'ed
@njit(fastmath=True, cache=True)
def markov_chain_simulation_nb(v_curr_state, v_draw, mtx_CTP0):
    v_next_state = np.empty_like(v_curr_state)
    for n in range(v_next_state.size):
        curr_s = v_curr_state[n]
        v_next_state[n] = digitize_scalar_nb(v_draw[n], mtx_CTP0[curr_s])
    return v_next_state

# call the above function, so that it is compiled and cached before running the benchmark runs below
markov_chain_simulation_nb(np.zeros(1, dtype=np.int64), np.zeros(1), np.array([[0.2]]))

# Numpy implementation
def markov_chain_simulation_np(v_curr_state, v_draw, mtx_CTP0):
    v_next_state = np.empty_like(v_curr_state)
    for s in range(0, mtx_CTP0.shape[0]):
        mask = v_curr_state == s
        v_next_state[mask] = np.digitize(v_draw[mask], mtx_CTP0[s])
    return v_next_state

## Transition Probability Matrix

* create a transition probability matrix using random numbers
* normalise each row to add up to one. 
* calculate the cumulative probability matrix. The last column, which consists of ones by definition, should be removed. 

In [10]:
# define transition probability matrix
num_state = 5
mtx_TP = np.random.uniform(0.0, 1.0, (num_state, num_state))
mtx_TP /= np.sum(mtx_TP, axis=1, keepdims=True)
mtx_CPT = np.cumsum(mtx_TP, axis=1)
mtx_CPT0 = mtx_CPT[:,:-1]

print('Transition Matrix')
print(mtx_TP)
print('Cumulative Transition Matrix')
print(mtx_CPT)

Transition Matrix
[[0.16835985 0.24909748 0.11624153 0.24116664 0.22513451]
 [0.28399115 0.01561099 0.29673091 0.26400873 0.13965822]
 [0.17578114 0.06803144 0.2445504  0.23297029 0.27866672]
 [0.13765799 0.22784075 0.18815866 0.21858196 0.22776065]
 [0.11716478 0.12659762 0.25013652 0.28501575 0.22108532]]
Cumulative Transition Matrix
[[0.16835985 0.41745733 0.53369886 0.77486549 1.        ]
 [0.28399115 0.29960214 0.59633305 0.86034178 1.        ]
 [0.17578114 0.24381259 0.48836299 0.72133328 1.        ]
 [0.13765799 0.36549874 0.55365739 0.77223935 1.        ]
 [0.11716478 0.24376241 0.49389893 0.77891468 1.        ]]


## Experiment 1
* 10 steps
* 1M scenarios

Time taken: 
naive >> numpy > numba

The numba verison is 
* over 50 time faster than the naive version
* over 3 time faster than the **numpy** version

In [16]:
num_step = 10
num_draw = 1000000

# initialise
v_init_state = np.random.randint(0, num_state, num_draw)
v_draw = np.random.uniform(0.0, 1.0, num_draw)

# simulation: naive
t0 = time.time()
v_next_naive = v_init_state.copy()
for n in range(num_step):
    v_next_naive = markov_chain_simulation_naive(v_next_naive, v_draw, mtx_CPT0)
print('naive: ' + str(time.time() - t0) + ' seconds')

# simulation: numba
t0 = time.time()
v_next_nb = v_init_state.copy()
for n in range(num_step):
    v_next_nb = markov_chain_simulation_nb(v_next_nb, v_draw, mtx_CPT0)
print('numba: ' + str(time.time() - t0) + ' seconds')

# simulation: numpy
t0 = time.time()
v_next_np = v_init_state.copy()
for n in range(num_step):
    v_next_np = markov_chain_simulation_np(v_next_np, v_draw, mtx_CPT0)
print('numpy: ' + str(time.time() - t0) + ' seconds')

print('result check: is naive equal to numba?', np.array_equal(v_next_naive, v_next_nb))
print('result check: is naive equal to numpy?', np.array_equal(v_next_naive, v_next_np))

naive: 7.585155248641968 seconds
numba: 0.11918163299560547 seconds
numpy: 0.39585041999816895 seconds
result check: is naive equal to numba? True
result check: is naive equal to numpy? True


## Experiment 2
* 10 steps
* 10M scenarios (10 times of experiment 1)
* only numpy and numba

Again, the numba verison is over 3 time faster than the **numpy** version

In [18]:
num_draw = 10000000
v_init_state = np.random.randint(0, num_state, num_draw)
v_draw = np.random.uniform(0.0, 1.0, num_draw)

# simulation: numba
t0 = time.time()
v_next_nb = v_init_state.copy()
for n in range(num_step):
    v_next_nb = markov_chain_simulation_nb(v_next_nb, v_draw, mtx_CPT0)
print('numba: ' + str(time.time() - t0) + ' seconds')

# simulation: numpy
t0 = time.time()
v_next_np = v_init_state.copy()
for n in range(num_step):
    v_next_np = markov_chain_simulation_np(v_next_np, v_draw, mtx_CPT0)
print('numpy: ' + str(time.time() - t0) + ' seconds')

print('result check: is numpy equal to numba?', np.array_equal(v_next_np, v_next_nb))

numba: 1.3647716045379639 seconds
numpy: 3.9955391883850098 seconds
result check: is numpy equal to numba? True
